## Importance Sampling Example

Author: Audrey Olivier 

Date: May 8, 2019

For importance sampling, the function must be written in a way that it can evaluate multiple samples at once.

In [1]:
from UQpy.Distributions import Uniform, JointInd
from UQpy.SampleMethods import IS
import time
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def log_Rosenbrock(x, param):
     return (-(100*(x[:, 1]-x[:, 0]**2)**2+(1-x[:, 0])**2)/param)
    
def Rosenbrock(x, param):
     return np.exp(-(100*(x[:, 1]-x[:, 0]**2)**2+(1-x[:, 0])**2)/param)

In [3]:
proposal = JointInd([Uniform(loc=-8, scale=16), Uniform(loc=-10, scale=60)])
print(proposal.get_params())

{'loc_0': -8, 'scale_0': 16, 'loc_1': -10, 'scale_1': 60}


### Run IS

In [4]:
t4 = time.time()
w = IS(log_pdf_target=log_Rosenbrock, args_target=(20, ), nsamples=10000, proposal=proposal)
#w = IS(pdf_target=Rosenbrock, args_target=(20, ), nsamples=10000, proposal=proposal)
t_IS = time.time()-t4
print(t_IS)

w.resample(nsamples=1000)
plt.plot(w.unweighted_samples[:,0], w.unweighted_samples[:,1],'gs',alpha=0.2)
print(w.unweighted_samples.shape)
plt.legend(['IS'])
plt.show()

NameError: name 'np' is not defined

### Run IS by adding samples: call the run method in a loop (one can also look at diagnostics)

In [ ]:
t4 = time.time()
w = IS(log_pdf_target=log_Rosenbrock, args_target=(20, ), proposal=proposal)
for nsamples in [5000, 5000, 5000, 5000]:
    w.run(nsamples)
    print(w.samples.shape)
    #IS_diagnostics(weights=w.weights, graphics=False)
t_IS = time.time()-t4
print(t_IS)

w.resample(nsamples=1000)
plt.plot(w.unweighted_samples[:,0],w.unweighted_samples[:,1],'gs',alpha=0.2)
plt.legend(['IS'])
plt.show()

Another example: sampling from a bivariate with copula dependence. Giving a random state enforces that results are the same for repeatability.

In [ ]:
from UQpy.Distributions import Normal, Gumbel, JointCopula
dist_true = JointCopula(marginals=[Normal(), Normal()], copula=Gumbel(theta=2.))
proposal = JointInd(marginals=[Normal(), Normal()])
sampler = IS(log_pdf_target=dist_true.log_pdf, nsamples=500, proposal=proposal, random_state=123)
print(sampler.samples.shape)
print(sampler.weights.shape)
print(np.round(sampler.samples[-5:], 4))